Source: https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

Datasource: https://www.kaggle.com/datasets/elvinrustam/books-dataset

In [1]:
!pip install gensim nltk pyLDAvis

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models
import pandas as pd
import os
import io

nltk.download('stopwords', force=True)
nltk.download('punkt', force=True)
nltk.download('punkt_tab')

In [22]:
nltk.data.find('tokenizers/punkt')

FileSystemPathPointer('/Users/sajjadislam/nltk_data/tokenizers/punkt')

In [23]:
nltk.data.find('corpora/stopwords')

FileSystemPathPointer('/Users/sajjadislam/nltk_data/corpora/stopwords')

In [24]:
nltk.data.path.append('/Users/sajjadislam/nltk_data')

In [26]:
# Read the CSV file into a DataFrame
df = pd.read_csv('../Data/BooksDataset.csv')

In [27]:
# Display the first few rows of the DataFrame
df.dropna(subset=['Description'], inplace=True)
df.sample(5)

,Title,Authors,Description,Category,Publisher,Publish Date,Price
69540,Rogue: A Novel,"By Steel, Danielle","Meet Maxine Williams, a dedicated doctor with ...","Fiction , Romance , Contemporary",Dell,"Monday, June 1, 2009",Price Starting at $4.99
27030,Say Yes to Life: A Book of Thoughts for Better...,"By Greenberg, Sidney",Brief essays discuss the endurance of human va...,"Religion , Inspirational",Random House Value Publishing,"Friday, June 1, 1984",Price Starting at $5.29
38374,Unlimited Access : An FBI Agent Inside the Cli...,"By Aldrich, Gary",Gary Aldrich was an FBI agent closing out his ...,"History , Modern , 20th Century",Regnery Publishing,"Sunday, March 1, 1998",Price Starting at $4.99
34974,I Can Count to Ten and Back Again,By Linda Hayward,Big Bird counts from one to ten as he stocks t...,NaN,Western Publishing,"Tuesday, January 1, 1985",Price Starting at $5.29
56501,Kid's Room: Ideas and Projects for Children's ...,"By Levy, Jennifer, Louie, Elaine, and Davey, D...",It's an age-old dilemma--how to create a styli...,"House & Home , General",Chronicle Books,"Tuesday, May 1, 2001",Price Starting at $10.99


In [28]:
df.dropna(subset=['Description'], inplace=True)
df.dropna(subset=['Category'], inplace=True)
df.shape

(65305, 7)

In [29]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Test preprocessing function
test_text = "This is a sample sentence for testing preprocessing."
stop_words = set(stopwords.words('english'))

try:
    words = word_tokenize(test_text.lower())
    processed_words = [word for word in words if word.isalpha() and word not in stop_words]
    print("Processed words:", processed_words)
except Exception as e:
    print(f"Error in preprocessing: {e}")

Processed words: ['sample', 'sentence', 'testing', 'preprocessing']


In [30]:
# Step 1: Preprocess Data
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    return [word for word in words if word.isalpha() and word not in stop_words]

processed_data = df['Description'].apply(preprocess)

In [15]:
# Create Dictionary and Corpus
dictionary = corpora.Dictionary(processed_data)
corpus = [dictionary.doc2bow(text) for text in processed_data]

# Step 2: LDA Model Training
lda_model = LdaModel(corpus, num_topics=65, id2word=dictionary, passes=15)

In [16]:
# Step 3: Visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

/Users/sajjadislam/opt/anaconda3/envs/py312_topic_modeling/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=4987) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/sajjadislam/opt/anaconda3/envs/py312_topic_modeling/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=4987) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/sajjadislam/opt/anaconda3/envs/py312_topic_modeling/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=4987) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/sajjadislam/opt/anaconda3/envs/py312_topic_modeling/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=4987) is multi-threaded, use 

In [17]:
# Step 4: Coherence Evaluation
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:', coherence_lda)


Coherence Score: 0.46693327033306253


3 topic = Coherence Score: 0.4019124985021872

5 topic = Coherence Score: 0.41309723401722753

8 topic = Coherence Score: 0.4705790575779403

10 topic = Coherence Score: 0.5009441512065992

12 topic = Coherence Score: 0.5341480811958599

15 topic = Coherence Score: 0.526427456404917

18 topic = Coherence Score: 0.5658992830252524

20 topic = Coherence Score: 0.5996165241455463

25 topic = Coherence Score: 0.612706784690936

30 topic = Coherence Score: 0.6262886789822056

35 topic = Coherence Score: 0.6487091875957105

40 topic = Coherence Score: 0.6571222299357229

45 topic = Coherence Score: 0.6489653946326334

50 topic = Coherence Score: 0.6589681585355589

55 topic = Coherence Score: 0.6644110944189812

60 topic = Coherence Score: 0.6607636903519484

65 topic = Coherence Score: 0.65579227219598

In [18]:
num_words = 5
topics = lda_model.print_topics(num_topics=-1, num_words=num_words)
for topic in topics:
    print(topic)

(0, '0.032*"travel" + 0.024*"training" + 0.023*"best" + 0.022*"program" + 0.021*"activities"')
(1, '0.053*"sarah" + 0.034*"bob" + 0.028*"harry" + 0.019*"stadium" + 0.019*"attractive"')
(2, '0.027*"history" + 0.011*"world" + 0.011*"historical" + 0.011*"work" + 0.011*"american"')
(3, '0.038*"max" + 0.034*"gifts" + 0.030*"farm" + 0.026*"tries" + 0.020*"accessories"')
(4, '0.069*"web" + 0.032*"system" + 0.025*"software" + 0.022*"data" + 0.022*"internet"')
(5, '0.180*"god" + 0.092*"faith" + 0.033*"prayer" + 0.030*"questions" + 0.023*"christ"')
(6, '0.030*"marriage" + 0.027*"ruth" + 0.022*"alice" + 0.022*"anne" + 0.020*"relationship"')
(7, '0.057*"jack" + 0.047*"henry" + 0.046*"king" + 0.036*"elizabeth" + 0.035*"queen"')
(8, '0.038*"heaven" + 0.030*"dating" + 0.029*"castle" + 0.025*"sons" + 0.022*"kelly"')
(9, '0.125*"new" + 0.077*"york" + 0.072*"times" + 0.046*"author" + 0.040*"bestselling"')
(10, '0.020*"american" + 0.016*"america" + 0.009*"history" + 0.009*"political" + 0.009*"years"')
(1

In [15]:
pip install git+https://github.com/rwalk/gsdmm.git


/Users/sajjadislam/opt/anaconda3/envs/py312_topic_modeling/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=30678) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


  Cloning https://github.com/rwalk/gsdmm.git to /private/var/folders/p3/s4n39zb50rb2l96w9n054ch80000gn/T/pip-req-build-y9495sb1
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /private/var/folders/p3/s4n39zb50rb2l96w9n054ch80000gn/T/pip-req-build-y9495sb1
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done
  Created wheel for gsdmm: filename=gsdmm-0.1-py3-none-any.whl size=4588 sha256=379fd2ded88367de72e60dd85899705c143797242f29b93316aaebc538397a45
  Stored in directory: /private/var/folders/p3/s4n39zb50rb2l96w9n054ch80000gn/T/pip-ephem-wheel-cache-oljw591c/wheels/f4/09/2a/066df0c20da7602aa8ba0c8157aedeef56199c5384f0aa1d09
Successfully built gsdmm
Note: you may need to restart the kernel to use updated packages.


In [16]:
from gsdmm import MovieGroupProcess
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [17]:
# Preprocess the Data
nltk.download('stopwords')
nltk.download('punkt')

def preprocess(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    return [word for word in words if word.isalpha() and word not in stop_words]

processed_data = [preprocess(text) for text in df['Comment']]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
# Create an instance of the GSDMM model
mgp = MovieGroupProcess(K=5, alpha=0.1, beta=0.1, n_iters=30)

# Fit the model on the data
vocab = set(x for doc in processed_data for x in doc)
n_terms = len(vocab)
y = mgp.fit(processed_data, n_terms)

# To see the topics for the first 10 documents
for i in range(5):
    print(f"Document: {processed_data[i]}, Topic: {mgp.choose_best_label(processed_data[i])}")

In stage 0: transferred 200 clusters with 5 clusters populated
In stage 1: transferred 104 clusters with 5 clusters populated
In stage 2: transferred 73 clusters with 5 clusters populated
In stage 3: transferred 60 clusters with 5 clusters populated
In stage 4: transferred 72 clusters with 5 clusters populated
In stage 5: transferred 61 clusters with 5 clusters populated
In stage 6: transferred 64 clusters with 5 clusters populated
In stage 7: transferred 61 clusters with 5 clusters populated
In stage 8: transferred 59 clusters with 5 clusters populated
In stage 9: transferred 57 clusters with 5 clusters populated
In stage 10: transferred 46 clusters with 5 clusters populated
In stage 11: transferred 46 clusters with 5 clusters populated
In stage 12: transferred 51 clusters with 5 clusters populated
In stage 13: transferred 36 clusters with 5 clusters populated
In stage 14: transferred 44 clusters with 5 clusters populated
In stage 15: transferred 38 clusters with 5 clusters populated


In [19]:
from collections import defaultdict
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

# Assuming mgp is your fitted GSDMM model and processed_data is your documents

# Find the dominant topic for each document
doc_topic = [mgp.choose_best_label(doc) for doc in processed_data]

# Word frequencies per topic
topic_word_freq = defaultdict(lambda: defaultdict(int))
for doc, topic in zip(processed_data, doc_topic):
    for word in doc:
        topic_word_freq[topic[0]][word] += 1

# Extract top N words for each topic
top_n = 5
top_words_per_topic = {}
for topic, word_freq in topic_word_freq.items():
    top_words = sorted(word_freq, key=word_freq.get, reverse=True)[:top_n]
    top_words_per_topic[topic] = top_words

# Create a dictionary and corpus for coherence calculation
dictionary = Dictionary(processed_data)
corpus = [dictionary.doc2bow(doc) for doc in processed_data]

# Using the top words for each topic to calculate coherence
coherence_model = CoherenceModel(topics=list(top_words_per_topic.values()), 
                                 texts=processed_data, 
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence_score = coherence_model.get_coherence()
print('Coherence Score:', coherence_score)


Coherence Score: 0.45296388042690194
